In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import joblib
import time
import itertools
import requests
from bs4 import BeautifulSoup
import time
import os
import pandas as pd
import codecs
from urllib.parse import urljoin
import numpy as np

In [2]:
df = pd.DataFrame([],columns=["職番","ページ数"])
for i in range(50):
    url="https://townwork.net/joSrchRsltList/?jc={0:03d}".format(i)
    res = requests.get(url)
    try:
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "lxml")
        divs = soup.select_one('#jsi-content-wrapper > div > div.job-cassette-lst-wrap > div.pager-wrap-bottom > div > div.pager-number-wrap > ol')
        m=re.findall(r"[0-9]{1,5}",str(divs))
        k=0
        for d in m:
            k=max(k,int(d))
        df=df.append({"職番":i,"ページ数":k},ignore_index=True)
    except:
        pass
    

In [3]:
df

,職番,ページ数
0,1,3969
1,2,3162
2,3,1594
3,4,79
4,5,295
5,7,11
6,8,64
7,9,205
8,10,2504
9,11,607


In [8]:
def joblib_url(i):
    url = 0
    new_list = list()
    for z in range(1,int(df['ページ数'][i])):
        urls = 'https://townwork.net/joSrchRsltList/?jc={0:03d}&page={1}'.format(df['職番'][i],z)
        res = requests.get(urls)
        try:
            res.raise_for_status()
            soup = BeautifulSoup(res.text, "lxml")
            divs = soup.find_all('div',class_='job-lst-box-wrap')
            for div in divs:
                try:
                    link = div.find('a')
                    href = link.attrs['href'] 
                    new_list.append(href)
                except:
                    pass
        except:
            pass
    return new_list
            

In [9]:
all_list = list()
resultList = joblib.Parallel(n_jobs=-1, verbose=3)( [joblib.delayed(joblib_url)(i) for i  in range(len(df['職番']))])
all_list.extend(resultList) 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  18 | elapsed: 28.6min remaining: 22.9min
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 82.0min finished


all_list = list()
a=range(1,50)
b=range(1,7000)
for i in a:
    try:
        for j in b:
            try:
                resultList =joblib_url(i,j)
                all_list.extend(resultList)
            except:
                pass
    except:
        pass

In [10]:
new_all_list = list(itertools.chain.from_iterable(all_list))

In [11]:
kekka = pd.DataFrame(new_all_list,columns=["url"])
kekka

,url
0,/detail/clc_0482114063/joid_Y006X9V2/?opf=A
1,/detail/clc_1424277001/?opf=PR1
2,/detail/clc_3959497253/
3,/detail/clc_1138989741/joid_Y006UUHG/
4,/detail/clc_3920495001/
...,...
716362,/detail/clc_3175529002/joid_Y006MAP2/
716363,/detail/clc_3594729001/
716364,/detail/clc_3890686001/joid_Y006SNFH/
716365,/detail/clc_3954603002/joid_Y006SFWL/


In [12]:
kekka.to_csv('townwork-url.csv')

In [4]:
kekk=pd.read_csv('townwork-url.csv')

In [5]:
kekka2=kekk.drop_duplicates(subset=['url'])

In [6]:
kekka2=kekka2.reset_index()

In [7]:
url_key = kekka2["url"]

In [50]:
def joblib_get_data(z):
    url = houjinnmei = shokushu = gyoukai = juusho = dennwabanngou = Web_url = pref = city =kyuuyo=page= ""
    new_list = list()
    url = "https://townwork.net/"+url_key[z]
    r = requests.get(url)
    try:
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "lxml")
        k=[]
        for u in range(20):
            try:
                juusho = soup.select_one("#js-detailArea > div:nth-child({}) > div > dl:nth-child(2) > dd > p ".format(u)).get_text()
                k.append(u)
            except:
                pass
        kk=max(k,default=10)
        try:
            houjinnmei = soup.select_one("#js-detailArea > div.job-detail-ttl-wrap > h3 > span.jsc-company-txt").get_text()
        except:
            houjinnmei = np.nan
        try:
            page=soup.select_one('#js-detailArea > div.job-detail-box-inner > div.job-detail-txt-wrap > div').get_text()
        except:
            pass
        try:
            kyuuyo=soup.select_one("#js-detailArea > div.job-detail-box-inner > div.job-detail-box-container > div.job-detail-tbl-wrap.job-detail-tbl-main-wrap > dl:nth-child(2) > dd > p > span").get_text()
        except:
            kyuuyo=""
        try:
            shokushu = soup.select_one("#js-detailArea > div.job-detail-box-inner > div.job-detail-box-container > div.job-detail-tbl-wrap.job-detail-tbl-main-wrap > dl:nth-child(1) > dd > p > span").get_text()
        except:
            shokushu = ""
        try:
            juusho = soup.select_one("#js-detailArea > div:nth-child({}) > div > dl:nth-child(3) > dd > p".format(kk)).get_text()
        except:
            juusho = ""
        try:
            dennwabanngou = soup.select_one("#js-detailArea > form > div > ul > li.detail-tel-box.jsc-phone-popup > p.detail-tel-ttl > span").get_text().split()
        except:
            dennwabanngou = ""
        try:
            gyoukai = soup.select_one("#js-detailArea > div:nth-child({}) > div > dl:nth-child(2) > dd > p".format(kk)).get_text()
        except:
            gyoukai = ""
        try:
            Web_url = soup.select_one("#js-detailArea > div:nth-child({}) > div > dl:nth-child(4) > dd > a".format(kk)).get_text()
        except:
            Web_url = ""
        try:
            lo=list()
            divs = soup.find_all('li',itemtype="http://data-vocabulary.org/Breadcrumb")
            for dv in divs:
                d =dv.find('span')
                href = d.getText('itemprop')
                lo.append(href)
            pref=lo[1]
            city=lo[2]
        except:
            pref = city =""
        new_list.append(houjinnmei)
        new_list.append(shokushu)
        new_list.append(kyuuyo)
        new_list.append(gyoukai)
        new_list.append(pref)
        new_list.append(city)
        new_list.append(juusho)
        new_list.append(dennwabanngou)
        new_list.append(Web_url)
        new_list.append(url)
        new_list.append(page)
    except:
        pass
    return new_list

In [96]:
c=round(len(url_key)/10000)

In [97]:
print(c)

62


In [98]:
all_list = list()
for n in range(c):
    resultList = joblib.Parallel(n_jobs=-1, verbose=3)([joblib.delayed(joblib_get_data)(i)  for i in range(n*10000,(n+1)*10000)])
    all_list.extend(resultList)

    

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 21.9min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed: 30.1min


KeyboardInterrupt: 

In [52]:
df_kekka = pd.DataFrame(all_list,columns=["法人名","媒体記載職種","給与情報","業種","都道府県","市区町村","勤務地住所","電話番号","webURL","ページurl",'ページタイトル'])
df_kekka['本社住所']=0
df_kekka['派遣・紹介企業']=0
df_kekka['施設名']=0
df_kekka['仕事内容']=df_kekka['媒体記載職種'].copy()

In [54]:
df_kekka

,法人名,媒体記載職種,給与情報,業種,都道府県,市区町村,勤務地住所,電話番号,webURL,ページurl,ページタイトル,本社住所,派遣・紹介企業,施設名,仕事内容
0,\r\n\t\t\t\t\t\t 大和ライフネクスト株式会社 ＜大和ハウスグループ＞\r\n...,[Ａ][Ｐ]火＆包丁の作業ナシ！社会人寮の調理補助,時給1013円＋交通費全額支給 ★土日休み★1日30食程度,マンション・ビル等の建物管理事業、他,東京,世田谷区,東京都港区赤坂5-1-33,[0120-30-2714/人材開発課(平日10-17時)],http://www.daiwalifenext.co.jp/,https://townwork.net//detail/clc_0482114063/jo...,(50代:女性)料理を作るのが苦手だった私。ここのお仕事は、食事の準備等の作業で調理業務がな...,0,0,0,[Ａ][Ｐ]火＆包丁の作業ナシ！社会人寮の調理補助
1,\r\n\t\t\t\t\t\t 株式会社 スポット\r\n\t\t\t\t\t\t,[Ａ][Ｐ]年末までの短期募集！(1)レジ・商品補充(2)調理補助,(1)時給1000円 (2)時給1100円,スーパーマーケットの運営 ＜ＨＰ＞https://www.spotgroup.co.jp,新潟,長岡市,長岡市新産3-6-12,"[050-1845-3259, /本部(佐々木・田中)]",https://www.spotgroup.co.jp,https://townwork.net//detail/clc_1424277001/?o...,★ 12/20頃～12/31までの短期バイト ★年末に向けてのお小遣い稼ぎにピッタリ!!未経...,0,0,0,[Ａ][Ｐ]年末までの短期募集！(1)レジ・商品補充(2)調理補助
2,\r\n\t\t\t\t\t\t むさしの森珈琲 ひたちなか西光地店＜198286＞\r\n...,[Ａ][Ｐ]オープニング/Ｗワークok/週0も、2hも◎カフェstaff,時給851円～/土日祝＋50円＜時給1127円有＞前給有/賄い,カフェ＆ダイニング,茨城,ひたちなか市,茨城県ひたちなか市東石川3088番地12,[0120-979-262/採用係(10:00-19:00)],https://crewrecruiting.skylark.co.jp/index.html,https://townwork.net//detail/clc_3959497253/,～穏やかな時間が流れる”お洒落カフェ”～2021年1月下旬、ひたちなか市に初オープン「スキマ...,0,0,0,[Ａ][Ｐ]オープニング/Ｗワークok/週0も、2hも◎カフェstaff
3,\r\n\t\t\t\t\t\t ゆず庵 静岡インター通店\r\n\t\t\t\t\t\t,[Ａ][Ｐ]ＮＥＷオープン！接客なし♪調理補助・洗い場,時給1000円～ ★高校生同額 ★22時以降/1250円～,急なお休みにも対応します★和食店,静岡,静岡市,静岡市駿河区南安倍3-251-1◎手ぶら面接ok！お気軽にどうぞ,[0120-314-086/平日9-21時、土日祝9-18時],http://www.monogatari.co.jp/,https://townwork.net//detail/clc_1138989741/jo...,新しいお店で♪カンタンな調理・サポート＜＜オープニングメンバー増員決定＞＞土日のみ・平日のみ...,0,0,0,[Ａ][Ｐ]ＮＥＷオープン！接客なし♪調理補助・洗い場
4,\r\n\t\t\t\t\t\t zukekura ＤＥＬＩ＆ＣＡＦＥ\r\n\t\t\t\...,[Ａ][Ｐ]オシャレ☆カフェスタッフ(キッチン補助),時給1000円～,カフェ＆デリカ,兵庫,宝塚市,兵庫県宝塚市湯本町2-5,"[050-1825-6031, 担当/野口]",,https://townwork.net//detail/clc_3920495001/,今年openした宝塚のリバーサイドカフェ☆ お料理やお菓子作りが好きな方歓迎!!一緒にお店を...,0,0,0,[Ａ][Ｐ]オシャレ☆カフェスタッフ(キッチン補助)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,\r\n\t\t\t\t\t\t Ｓ・ＴＥＣ株式会社\r\n\t\t\t\t\t\t,[Ａ][Ｐ]主婦活躍中!!(1)食器洗浄業務(2)厨房清掃,時給1100円 ※22～翌5時/時給1375円(深夜込),業務請負業,石川,金沢市,(東京本社)新宿区岩戸町14番地 神楽坂不二ビル,[0120-278-088＜受付＞9～19時/採用係],http://www.stec-ltd.com,https://townwork.net//detail/clc_2861254001/jo...,金沢駅西口目の前!!徒歩1分で通勤ラク2♪週2日～スキマ時間を有効活用できる◎扶養内勤務ＯＫ...,0,0,0,[Ａ][Ｐ]主婦活躍中!!(1)食器洗浄業務(2)厨房清掃
96,\r\n\t\t\t\t\t\t 焼肉 なかむら\r\n\t\t\t\t\t\t,[Ａ][Ｐ]あの有名焼肉店!?(1)ホールＳtaff(2)キッチンＳtaff,時給950円～/22時以降:時給1188円 ◇交通費支給有,焼肉屋,香川,高松市,(A)香川町大野444-14 (B)屋島西町2484-4 (C)西の丸町13-7,[上記参照/各店長(受付:15時～)],,https://townwork.net//detail/clc_2182224001/,失敗なんて気にしなくてＯＫ～～！！→→そんな店長がいるから働きやすい→→どうせなら楽しみな...,0,0,0,[Ａ][Ｐ]あの有名焼肉店!?(1)ホールＳtaff(2)キッチンＳtaff
97,\r\n\t\t\t\t\t\t 株式会社アルファ 日の出食堂\r\n\t\t\t\t\t\t,[Ａ][Ｐ]おしゃれな定食屋♪賄い付☆(1)ホール(2)キッチン,時給900円＋交規給※スキルに応じて時給アップ制度あり,定食屋、飲食店,福岡,福岡市,福岡市博多区博多駅南1-14-3 アミューゼ博多1Ｆ,[092-483-2227(受付:平日9～18時)],http://hinodesyokudo.jp/,https://townwork.net//detail/clc_3234261002/,主婦(夫)さん、学生さん大歓迎♪ 博多駅チカで通いやすい好立地☆頑張りは昇給でしっかり...,0,0,0,[Ａ][Ｐ]おしゃれな定食屋♪賄い付☆(1)ホール(2)キッチン
98,\r\n\t\t\t\t\t\t セントラルキッチン 高円寺 \r\n\t\t\t\t\t\t,[Ａ][Ｐ]セントラルキッチンの調理スタッフ＊前払いＯＫ,時給1050円以上 【12/30～1/3は時給1300円！】,飲食業,東京,杉並区,杉並区高円寺南4-49-1,[03-5364-5321/お問い合わせ(14-17時)],http://www.apcompany.jp/recruit/staff/,https://townwork.net//detail/clc_1999867008/jo...,選べる勤務時間♪シフト相談に乗ります◎学校帰りに「3時間だけ働きたい」も歓迎!!セントラルキ...,0,0,0,[Ａ][Ｐ]セントラルキッチンの調理スタッフ＊前払いＯＫ


In [56]:
df_kekka.to_csv('townworkkekka.csv')

In [2]:
df_kekka=pd.read_csv('townworkkekka.csv')

In [3]:
df_ka=df_kekka.copy()
df_ka=df_ka.dropna(subset=['法人名'])
df_ka=df_ka.reset_index()

In [4]:
df_ka.loc[df_ka['都道府県'] == '大阪','a'] ="府"
df_ka.loc[df_ka['都道府県']=='京都','a']='府'
df_ka.loc[df_ka['都道府県']=='東京',"a"]='都'
df_ka.loc[~(df_ka['都道府県']=='大阪') & ~(df_ka['都道府県']=='京都')&~(df_ka['都道府県']=='東京')&~(df_ka['都道府県']=='北海道'),'a']='県'

In [5]:
df_ka['都道府県']=df_ka['都道府県']+df_ka['a'].fillna('')

In [6]:
df_ka['法人名']=df_ka['法人名'].str.replace("\r\n\t\t\t\t\t\t","")

In [7]:
df_ka['電話番号']=df_ka['電話番号'].astype(str).str.extract('([0-9]{2,4}-[0-9]{2,4}-[0-9]{2,4})')

In [8]:
df_ka['電話番号'].astype(str).str.extract('([0-9]{2,4}-[0-9]{2,4}-[0-9]{2,4})')

,0
0,0120-30-2714
1,050-1845-3259
2,0120-979-262
3,0120-314-086
4,050-1825-6031
...,...
94,0120-278-088
95,NaN
96,092-483-2227
97,03-5364-5321


In [9]:
import jaconv
#法人名をクレンジング：全て全角に変換。法人番号リストの法人名にもあとで同じ処理を加える。
#全角に直す関数を定義
def normalize(s):
    s = str(s)
    s = jaconv.z2h(s,digit=True,ascii=True)
    return s

In [10]:
df_ka['媒体記載職種']=df_ka['媒体記載職種'].apply(normalize)

In [11]:
df_ka['A']=df_ka['媒体記載職種'].astype(str).str.extract('([A])')

In [12]:
df_ka['P']=df_ka['媒体記載職種'].astype(str).str.extract('([P])')

In [13]:
df_ka['社']=df_ka['媒体記載職種'].astype(str).str.extract('([社])')

In [14]:
df_ka['契']=df_ka['媒体記載職種'].astype(str).str.extract('([契])')

In [15]:
kwd_ser=df_ka.columns[-4:]

In [16]:
kwd_ser

Index(['A', 'P', '社', '契'], dtype='object')

In [17]:
df_ka['雇用区分']=""
for kwd in kwd_ser:
    df_ka['雇用区分']=df_ka['雇用区分']+','+df_ka[kwd].fillna('')

In [18]:
df_ka['雇用区分']=df_ka['雇用区分'].str.replace(r"\,+", " ").str.strip()

In [19]:
df_ka['雇用区分']=df_ka['雇用区分'].str.replace('A','アルバイト')
df_ka['雇用区分']=df_ka['雇用区分'].str.replace('P','パート')
df_ka['雇用区分']=df_ka['雇用区分'].str.replace('社','正社員')
df_ka['雇用区分']=df_ka['雇用区分'].str.replace('契','契約社員')

In [20]:
df_ka=df_ka.drop(['a','A','P','社','契','webURL'],axis=1)

In [29]:
df_ka['給与情報実験']=df_ka['給与情報'].copy()
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('万円','0000円')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('千円','000円')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('万','')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('(','あ')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace(')','あ')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace(',','')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('：','')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('.','')
df_ka['給与数字']=''
df_ka['給与区分']=''
df_ka['給与上限']=0
df_ka['給与下限']=float("inf")
df_ka['給なし'] = df_ka['給与情報実験'].str.match('(.*給.+円)', na=False)
df_ka['出来高']=df_ka['給与情報実験'].astype(str).str.extract('(出来高)')
df_ka['歩合']=df_ka['給与情報実験'].astype(str).str.extract('(歩合)')
df_ka['給あり']=df_ka['給与情報実験'].astype(str).str.extract('(.*給.+円)')
for i in range (len(df_ka['出来高'])):
    if df_ka['出来高'][i]=='出来高':
        df_ka['給与区分'][i]='出来高'
        df_ka['給与上限'][i]=df_ka['給与下限'][i]=np.nan
    else:
        if df_ka['給なし'][i]==False :
            n=str(df_ka['給与情報実験'][i])
            j=re.sub('あ.+?あ','', n)
            j=re.sub('[0-9]{1,2}時',"",j)
            j=re.sub('[0-9]{1,3}%|[0-9]{1,3}プラス|プラス[0-9]{1,3}|[0-9]{1,3}UP',"",j)
            j=re.sub('交通.+[0-9]+?円',"",j)
            df_ka['給与情報実験'][i]=j
#    m=re.findall(r'.給([0-9]+)円|～([0-9]+)円|([0-9]+)～円',df_ka['e'][i])
            m=re.findall(r'([0-9]{4,9})',df_ka['給与情報実験'][i])
            df_ka['給与数字'][i]=m
            h=re.findall(r'(.給)',df_ka['給与情報実験'][i])
            df_ka['給与区分'][i]=h
        else:
            n=str(df_ka['給あり'][i])
            j=re.sub('あ.+?あ','', n)
            j=re.sub('[0-9]{1,2}時',"",j)
            j=re.sub('[0-9]{1,3}%|[0-9]{1,3}プラス|プラス[0-9]{1,3}|[0-9]{1,3}UP',"",j)
            j=re.sub('交通.+[0-9]+?円',"",j)
            df_ka['給あり'][i]=j
#    m=re.findall(r'.給([0-9]+)円|～([0-9]+)円|([0-9]+)～円',df_ka['e'][i])
            m=re.findall(r'([0-9]{3,9}?)円',df_ka['給あり'][i])
            df_ka['給与数字'][i]=m
            h=re.findall(r'(.給)',df_ka['給あり'][i])
            df_ka['給与区分'][i]=h

for i in range(len(df_ka['給与区分'])):
    if  df_ka['出来高'][i]!='出来高' :
        flat =",".join(map(str, df_ka['給与区分'][i]))
        sp=flat.split(',')
        kore=set(sp)
        kore.discard('支給')
        kore.discard('合給')
        kore.discard('、給')
        df_ka['給与区分'][i]=kore
        if df_ka['歩合'][i]=='歩合':
            df_ka['給与区分'][i].add('歩合')
        df_ka['給与区分'][i]=list(df_ka['給与区分'][i])
for i in range(len(df_ka['給与数字'])):
    if  df_ka['出来高'][i]!='出来高':
        for j in df_ka["給与数字"][i]:
            k=int(j)
            if k>df_ka['給与上限'][i]:
                df_ka['給与上限'][i]=k
            if k<df_ka['給与下限'][i]:
                df_ka['給与下限'][i]=k
  
   
for i in range(len(df_ka['歩合'])):
    if df_ka['歩合'][i]=='歩合':
        if df_ka['給与区分'][i][0]=='':
            df_ka['給与上限'][i]=df_ka['給与下限'][i]=np.nan
    else:
        if df_ka['給与区分'][i][0]=='':
            df_ka['給与区分']='日給'

C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A valu

In [30]:
df_ka.columns

Index(['index', 'Unnamed: 0', '法人名', '媒体記載職種', '給与情報', '業種', '都道府県', '市区町村',
       '勤務地住所', '電話番号', 'ページurl', 'ページタイトル', '本社住所', '派遣・紹介企業', '施設名', '仕事内容',
       '雇用区分', '給与情報実験', '給与数字', '給与区分', '給与上限', '給与下限', '給なし', '出来高', '給あり',
       '歩合'],
      dtype='object')

In [31]:
df2=df_ka.drop(['index','給なし', '出来高', '給あり', '給与情報実験', '給与数字'],axis=1)

In [32]:
dfrust=df2.reindex(columns=['法人名','施設名','勤務地住所','本社住所','都道府県','市区町村','雇用区分','ページタイトル','媒体記載職種','業種','電話番号','給与情報','給与区分','給与下限','給与上限','仕事内容','取得日','媒体名','派遣・紹介企業','ページurl'])

In [33]:
import datetime
dfrust['取得日']=datetime.date.today( )
dfrust['媒体名']='タウンワーク'

In [34]:
dfrust

,法人名,施設名,勤務地住所,本社住所,都道府県,市区町村,雇用区分,ページタイトル,媒体記載職種,業種,電話番号,給与情報,給与区分,給与下限,給与上限,仕事内容,取得日,媒体名,派遣・紹介企業,ページurl
0,大和ライフネクスト株式会社 ＜大和ハウスグループ＞,0,東京都港区赤坂5-1-33,0,東京都,世田谷区,アルバイト パート 正社員,(50代:女性)料理を作るのが苦手だった私。ここのお仕事は、食事の準備等の作業で調理業務がな...,[A][P]火&包丁の作業ﾅｼ!社会人寮の調理補助,マンション・ビル等の建物管理事業、他,0120-30-2714,時給1013円＋交通費全額支給 ★土日休み★1日30食程度,[時給],1013.0,1013,[Ａ][Ｐ]火＆包丁の作業ナシ！社会人寮の調理補助,2020-12-16,タウンワーク,0,https://townwork.net//detail/clc_0482114063/jo...
1,株式会社 スポット,0,長岡市新産3-6-12,0,新潟県,長岡市,アルバイト パート,★ 12/20頃～12/31までの短期バイト ★年末に向けてのお小遣い稼ぎにピッタリ!!未経...,[A][P]年末までの短期募集!(1)ﾚｼﾞ･商品補充(2)調理補助,スーパーマーケットの運営 ＜ＨＰ＞https://www.spotgroup.co.jp,050-1845-3259,(1)時給1000円 (2)時給1100円,[時給],1000.0,1100,[Ａ][Ｐ]年末までの短期募集！(1)レジ・商品補充(2)調理補助,2020-12-16,タウンワーク,0,https://townwork.net//detail/clc_1424277001/?o...
2,むさしの森珈琲 ひたちなか西光地店＜198286＞,0,茨城県ひたちなか市東石川3088番地12,0,茨城県,ひたちなか市,アルバイト パート,～穏やかな時間が流れる”お洒落カフェ”～2021年1月下旬、ひたちなか市に初オープン「スキマ...,[A][P]ｵｰﾌﾟﾆﾝｸﾞ/Wﾜｰｸok/週0も､2hも◎ｶﾌｪstaff,カフェ＆ダイニング,0120-979-262,時給851円～/土日祝＋50円＜時給1127円有＞前給有/賄い,[時給],851.0,1127,[Ａ][Ｐ]オープニング/Ｗワークok/週0も、2hも◎カフェstaff,2020-12-16,タウンワーク,0,https://townwork.net//detail/clc_3959497253/
3,ゆず庵 静岡インター通店,0,静岡市駿河区南安倍3-251-1◎手ぶら面接ok！お気軽にどうぞ,0,静岡県,静岡市,アルバイト パート,新しいお店で♪カンタンな調理・サポート＜＜オープニングメンバー増員決定＞＞土日のみ・平日のみ...,[A][P]NEWｵｰﾌﾟﾝ!接客なし♪調理補助･洗い場,急なお休みにも対応します★和食店,0120-314-086,時給1000円～ ★高校生同額 ★22時以降/1250円～,[時給],1000.0,1250,[Ａ][Ｐ]ＮＥＷオープン！接客なし♪調理補助・洗い場,2020-12-16,タウンワーク,0,https://townwork.net//detail/clc_1138989741/jo...
4,zukekura ＤＥＬＩ＆ＣＡＦＥ,0,兵庫県宝塚市湯本町2-5,0,兵庫県,宝塚市,アルバイト パート,今年openした宝塚のリバーサイドカフェ☆ お料理やお菓子作りが好きな方歓迎!!一緒にお店を...,[A][P]ｵｼｬﾚ☆ｶﾌｪｽﾀｯﾌ(ｷｯﾁﾝ補助),カフェ＆デリカ,050-1825-6031,時給1000円～,[時給],1000.0,1000,[Ａ][Ｐ]オシャレ☆カフェスタッフ(キッチン補助),2020-12-16,タウンワーク,0,https://townwork.net//detail/clc_3920495001/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Ｓ・ＴＥＣ株式会社,0,(東京本社)新宿区岩戸町14番地 神楽坂不二ビル,0,石川県,金沢市,アルバイト パート,金沢駅西口目の前!!徒歩1分で通勤ラク2♪週2日～スキマ時間を有効活用できる◎扶養内勤務ＯＫ...,[A][P]主婦活躍中!!(1)食器洗浄業務(2)厨房清掃,業務請負業,0120-278-088,時給1100円 ※22～翌5時/時給1375円(深夜込),[時給],1100.0,1375,[Ａ][Ｐ]主婦活躍中!!(1)食器洗浄業務(2)厨房清掃,2020-12-16,タウンワーク,0,https://townwork.net//detail/clc_2861254001/jo...
95,焼肉 なかむら,0,(A)香川町大野444-14 (B)屋島西町2484-4 (C)西の丸町13-7,0,香川県,高松市,アルバイト パート,失敗なんて気にしなくてＯＫ～～！！→→そんな店長がいるから働きやすい→→どうせなら楽しみな...,[A][P]あの有名焼肉店!?(1)ﾎｰﾙStaff(2)ｷｯﾁﾝStaff,焼肉屋,NaN,時給950円～/22時以降:時給1188円 ◇交通費支給有,[時給],950.0,1188,[Ａ][Ｐ]あの有名焼肉店!?(1)ホールＳtaff(2)キッチンＳtaff,2020-12-16,タウンワーク,0,https://townwork.net//detail/clc_2182224001/
96,株式会社アルファ 日の出食堂,0,福岡市博多区博多駅南1-14-3 アミューゼ博多1Ｆ,0,福岡県,福岡市,アルバイト パート,主婦(夫)さん、学生さん大歓迎♪ 博多駅チカで通いやすい好立地☆頑張りは昇給でしっかり...,[A][P]おしゃれな定食屋♪賄い付☆(1)ﾎｰﾙ(2)ｷｯﾁﾝ,定食屋、飲食店,092-483-2227,時給900円＋交規給※スキルに応じて時給アップ制度あり,[時給],900.0,900,[Ａ][Ｐ]おしゃれな定食屋♪賄い付☆(1)ホール(2)キッチン,2020-12-16,タウンワーク,0,https://townwork.net//detail/clc_3234261002/
97,セントラルキッチン 高円寺,0,杉並区高円寺南4-49-1,0,東京都,杉並区,アルバイト パート,選べる勤務時間♪シフト相談に乗ります◎学校帰りに「3時間だけ働きたい」も歓迎!!セントラルキ...,[A][P]ｾﾝﾄﾗﾙｷｯﾁﾝの調理ｽﾀｯﾌ*前払いOK,飲食業,03-5364-5321,時給1050円以上 【12/30～1/3は時給1300円！】,[時給],1050.0,1300,[Ａ][Ｐ]セントラルキッチンの調理スタッフ＊前払いＯＫ,2020-12-16,タウンワーク,0,https://townwork.net//detail/clc_1999867008/jo...


In [ ]:
dfrust.to_csv('タウンワークスクレイピング_{}.csv'.format(datetime.date.today()))